### Sprint 6: Capitulo 1.1 - Diccionarios: recorrer diccionarios en bucle

In [ ]:
import numpy as np

financial_info = {
    'American Express': 93.23,
    'Boeing': 178.44,
    'Coca-Cola': 45.15,
    'Walt Disney': 119.34,
    'Nike': 97.99,
    'JPMorgan':96.27,
    'Walmart': 130.68 
}

for key in financial_info.keys():
    print(key)

print()

for value in financial_info.values():
    print(value)

print()

total_value = 0 

for value in financial_info.values():
    total_value += value

print('Total value: ', np.ceil(total_value))
print()

for key, value in financial_info.items():
    print(key, value)

total_value_2 = 0

for key, value in financial_info.items():
    total_value_2 += value

print()
print(total_value_2)

### Sprint 6: Capitulo 1.2 - Diccionarios: estructuras de datos anidadas con diccionarios

In [ ]:
bus_schedule = {
    '72': ['8:00', '12:00', '17:30'],
    '26': ['9:30', '15:00'],
    '17': ['7:30', '12:30', '15:30']
}

# iteramos sobre las claves y los valores del diccionario
for route, times in bus_schedule.items():
    # iteramos sobre los valores de la lista
    for time in times:
        # mostramos la ruta y su horario correspondiente
	      print(f"Ruta {route} - Hora {time}")

print()

movies_table = [
    ['The Shawshank Redemption', 'USA', 'drama', 1994, 142, 9.111],
    ['The Godfather', 'USA', 'drama, crime', 1972, 175, 8.730],
    ['The Dark Knight', 'USA', 'fantasy, action, thriller', 2008, 152, 8.499]
]

print(movies_table[2][3])

print()

movies_table = [
    {'movie_name':'The Shawshank Redemption', 'country':'USA', 'genre':'drama', 'year':1994, 'duration':142, 'rating':9.111},
    {'movie_name':'The Godfather', 'country':'USA', 'genre':'drama, crime', 'year':1972, 'duration':175, 'rating':8.730},
    {'movie_name':'The Dark Knight', 'country':'USA', 'genre':'fantasy, action, thriller', 'year':2008, 'duration':152, 'rating':8.499}
]

# ahora accedemos a la columna por su nombre:
print(movies_table[2]['movie_name'])

### Sprint 6: Capitulo 1.3 - Diccionarios: procesar listas de diccionarios

In [ ]:
order = [
    {
        'item': 'Margherita pizza',
        'category': 'pizza',
        'quantity': 2,
        'price': 9
    },
    {
        'item': 'Ham pizza',
        'category': 'pizza',
        'quantity': 1,
        'price': 12
    },
    {
        'item': 'Pepsi 1 l',
        'category': 'beverages',
        'quantity': 3,
        'price': 2
    }
]

# variable del precio total del pedido
total_price = 0 

# iterar sobre cada diccionario de la lista
for item in order: 
# añadir a la variable el precio del elemento multiplicado por la cantidad
    total_price += item['price'] * item['quantity']

print(total_price)
print()

# variable para almacenar el resultado
filtered_order = [] 

for item in order: # iterar sobre cada diccionario de la lista
	if item['category'] == 'pizza': # si la categoría es pizza...
		filtered_order.append(item) # agregamos el diccionario a la lista filtered_order

# mostramos la lista filtrada de diccionarios
print(filtered_order)


In [ ]:
order = [
    {
        'item': 'Margherita pizza',
        'category': 'pizza',
        'quantity': 2,
        'comment': 'Add extra cheese please!',
        'price': 9
    },
    {
        'item': 'Ham pizza',
        'category': 'pizza',
        'quantity': 1,
        'comment': '',
        'price': 12
    },
    {
        'item': 'Pepsi 1 l',
        'category': 'beverages',
        'quantity': 3,
        'comment': '',
        'price': 2
    },
    {
        'item': 'Apple juice 0.5 l',
        'category': 'beverages',
        'quantity': 1,
        'comment': '',
        'price': 1
    },
    {
        'item': 'Croissant with cheese',
        'category': 'baked foods',
        'quantity': 2,
        'comment': '',
        'price': 1
    }
]

total_cost = 0

# empieza a escribir tu bucle for aquí
for item in order:
    if item['category'] == 'pizza':
        total_cost += item['price'] * item['quantity']

print(total_cost)

### Sprint 6: Capitulo 1.4 - Funciones: parámetros y valores predeterminados

In [ ]:
# esta función imprime las tablas filtradas. No modificar
def print_movie_info(data):
    for movie in data:
        print(movie)


# define tu función filter_by_genre() aquí
def filter_by_genre(data, genre='drama'):
    filtered_list = []

    for row in data:
        if genre in row[3]:
            filtered_list.append(row)

    return filtered_list

movies_info = [
    ['The Shawshank Redemption', 'USA', 1994, 'drama', 142, 9.111],
    ['The Godfather', 'USA', 1972, 'drama, crime', 175, 8.730],
    ['The Dark Knight', 'USA', 2008, 'fantasy, action, thriller', 152, 8.499],
    ["Schindler's List", 'USA', 1993, 'drama', 195, 8.818],
    ['The Lord of the Rings: The Return of the King', 'New Zealand', 2003, 'fantasy, adventure, drama', 201, 8.625],
    ['Pulp Fiction', 'USA', 1994, 'thriller, comedy, crime', 154, 8.619],
    ['The Good, the Bad and the Ugly', 'Italy', 1966, 'western', 178, 8.521],
    ['Fight Club', 'USA', 1999, 'thriller, drama, crime', 139, 8.644],
    ['Harakiri', 'Japan', 1962, 'drama, action, history', 133, 8.106],
    ['Good Will Hunting', 'USA', 1997, 'drama, romance', 126, 8.077]
]

# a continuación ejecutamos dos funciones para obtener resultados. No las modifiques
movies_filtered = filter_by_genre(movies_info)
print_movie_info(movies_filtered)

### Sprint 6: Capitulo 1.5 - Funciones: valores de retorno

In [ ]:
def omelet(eggs_number=2):
    if eggs_number > 0:
        return '¡El omelet está listo! Huevos utilizados: ' + str(eggs_number)
    else:
        return '¿Cómo puedo hacer un omelet sin huevos?'
    print('Nunca podré mostrar el resultado :(')


print(omelet(3))
print(omelet(0))
print()

def find_area_and_perim(side1, side2):
    area = side1 * side2
    perimeter = 2 * (side1 + side2)
    return area, perimeter

# descomprime el resultado de la función
rec_area, rec_perimeter = (find_area_and_perim(7, 3))

print(f'El área del rectángulo es {rec_area}, el perímetro es {rec_perimeter}')

### Sprint 6: Capitulo 1.6 - Pandas: el objeto Series

In [ ]:
import pandas as pd

df = pd.read_csv('E:\\datasets\\music_log_chpt_11.csv')

part_df = df[['genre', 'Artist']]
print('2 columnas tienen el tipo Data Frame',type(part_df))
print('La tabla tiene tipo Data Frame',type(df))

part_df = df['Artist']
print('1 columna es de tipo Series', type(part_df))

In [ ]:
import pandas as pd

df = pd.read_csv('E:\\datasets\\music_log_chpt_11.csv')

part_df = df['Artist']

print('Name',part_df.name)
print('Size',part_df.size)
print('Value', part_df[0])
print(part_df[[0,2,5]])
part_df.head(10)

In [ ]:
import pandas as pd

df = pd.read_csv('E:\\datasets\\music_log_chpt_11.csv')

total_play = df['total play']
lower_20 = total_play < 20

print(lower_20)
print()

series_lower20_only = total_play[lower_20]

print(series_lower20_only)
print()

df_lower20_only = df[lower_20]

print(df_lower20_only)


In [ ]:
import pandas as pd

df = pd.read_csv('E:\\datasets\\music_log_chpt_11.csv')

tracks = df['track']
track_check = tracks == 'Andrew Paul Woodworth'

print(track_check)

df['track'] == 'Andrew Paul Woodworth'

In [ ]:
import pandas as pd

df = pd.read_csv('E:\\datasets\\music_log_chpt_11.csv')

print(df[df['genre'] == 'pop'].head())

genre = df['genre']
pop_genre_check = genre == 'pop'
pop_df = df[pop_genre_check]

print(pop_df.head())

### Sprint 6: Capitulo 1.7 - Pandas: estadística descriptiva

In [ ]:
import pandas as pd

df = pd.read_csv('E:\\datasets\\music_log_chpt_11.csv')

print(df['total play'].max())
print(df[df['total play'] == df['total play'].max()])

In [79]:
import pandas as pd

df = pd.read_csv('E:\\datasets\\music_log_processed.csv')

pop_tracks = df[df['genre'] == 'pop']
pop_tracks = pop_tracks[pop_tracks['total_play'] > 30]

pop_max = pop_tracks['total_play'].max()
pop_mean = pop_tracks['total_play'].mean()
pop_median = pop_tracks['total_play'].median()
 
print('Max pop: ', pop_max)
print('Mean pop', pop_mean)
print('Pop median', pop_median)
print()

df_drop_skip = df[df['total_play'] > 30]

print('Min 30: ', df_drop_skip['total_play'].min())
print()

print(df_drop_skip[df_drop_skip['total_play'] == df_drop_skip['total_play'].min()])
print()

print('Median: ', df_drop_skip['total_play'].median())
print('Mean:', df_drop_skip['total_play'].mean())

Max pop:  1158.0280450000016
Mean pop 186.25846123970464
Pop median 197.996

Min 30:  30.000497

        user_id  total_play       artist genre                         track
56351  95B68844   30.000497  Bob Baldwin  jazz  Can You Feel It? (Interlude)

Median:  201.96400000000003
Mean: 206.13374546931612
